# 1 Let's explore the free throw play-by-play data
## Is it true that player's shoot better on the 3rd shot when they shoot better in the first two shots?

Let's import the python package that allows us to use Stata with Jupyter

In [2]:
import ipystata

## 2.1 Load the Free Trhow data, and keep only three-shot free-throws.

In [12]:
%%stata
global datadir "data"

clear
use "$datadir\\FreeThrow.dta",replace

*Keep only 3 shot freet throws
keep if outof==3 //keep only triples


(394,733 observations deleted)



## 2.2 Generate needed variables
in particular the shooting percentage the first 2 shots (0, 0.5, or 1).

In [13]:
%%stata

*Generate Unique player id, and unique game id
egen sid=group(playerteam)
sort season gid playerteam
egen uid=group(uniquegid playerteam)

*Creat variable FG% in the previous 2 shots for each triple
sort sid uid linenumber
by sid:gen fg_last2=(make[_n-1]+make[_n-2])/2 if number==3

keep if fg_last2!=.


(4,626 missing values generated)

(4,626 observations deleted)



## 2.3 Run fixed effects model of $\text{hit}_3=\beta_0+\beta_1\frac{\text{hit}_1+\text{hit}_2}{2}$, i.e. the effect of the fg% in the previous 2 shots on the 3rd shot

### Notice the effect is substantial, and nearly significant

In [14]:
%%stata
preserve
    collapse make, by(playerteam sid fg_last2)
    xtset sid
    xtreg make fg_last2, fe robust
restore


       panel variable:  sid (unbalanced)

Fixed-effects (within) regression               Number of obs     =        790
Group variable: sid                             Number of groups  =        503

R-sq:                                           Obs per group:
     within  = 0.0095                                         min =          1
     between = 0.0075                                         avg =        1.6
     overall = 0.0069                                         max =          3

                                                F(1,502)          =       2.65
corr(u_i, Xb)  = 0.0132                         Prob > F          =     0.1043

                                  (Std. Err. adjusted for 503 clusters in sid)
------------------------------------------------------------------------------
             |               Robust
        make |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+---------------------------------------------------

A bit redudandant, but here is a  matched pairs t-test gives similar results, Pr(hit| 2hits) vs. Pr(hit|0 hits):

In [9]:
%%stata
preserve 
    collapse make, by(playerteam sid fg_last2)
    drop if fg_last2==.5
    sort sid
    by sid:egen numbersobs=count(make)
    keep if numbersobs==2
    sort sid fg_last2
    by sid:gen fg_2=make[2]
    by sid:gen fg_0=make[1]
    ttest fg_2=fg_0 if fg_last2==0
restore



(301 observations deleted)

(349 observations deleted)

Paired t test
------------------------------------------------------------------------------
Variable |     Obs        Mean    Std. Err.   Std. Dev.   [95% Conf. Interval]
---------+--------------------------------------------------------------------
    fg_2 |      70    .8473139    .0268263    .2244449    .7937969    .9008309
    fg_0 |      70    .7821429    .0457943    .3831424    .6907858    .8734999
---------+--------------------------------------------------------------------
    diff |      70     .065171    .0513224    .4293939   -.0372143    .1675564
------------------------------------------------------------------------------
     mean(diff) = mean(fg_2 - fg_0)                               t =   1.2698
 Ho: mean(diff) = 0                              degrees of freedom =       69

 Ha: mean(diff) < 0           Ha: mean(diff) != 0           Ha: mean(diff) > 0
 Pr(T < t) = 0.8958         Pr(|T| > |t|) = 0.2084         

A matched pairs t-test, Pr(hit| 2hits) vs. Pr(hit|1 hit):

In [15]:
%%stata
preserve 
    collapse make, by(playerteam sid fg_last2)
    drop if fg_last2==0
    sort sid
    by sid:egen numbersobs=count(make)
    keep if numbersobs==2
    sort sid fg_last2
    by sid:gen fg_2=make[2]
    by sid:gen fg_1=make[1]
    ttest fg_2=fg_1 if fg_last2==.5
restore


(88 observations deleted)

(278 observations deleted)

Paired t test
------------------------------------------------------------------------------
Variable |     Obs        Mean    Std. Err.   Std. Dev.   [95% Conf. Interval]
---------+--------------------------------------------------------------------
    fg_2 |     212    .8615923    .0158101    .2301979    .8304264    .8927582
    fg_1 |     212    .8283464    .0213914    .3114635    .7861782    .8705147
---------+--------------------------------------------------------------------
    diff |     212    .0332459    .0285124    .4151474   -.0229599    .0894516
------------------------------------------------------------------------------
     mean(diff) = mean(fg_2 - fg_1)                               t =   1.1660
 Ho: mean(diff) = 0                              degrees of freedom =      211

 Ha: mean(diff) < 0           Ha: mean(diff) != 0           Ha: mean(diff) > 0
 Pr(T < t) = 0.8775         Pr(|T| > |t|) = 0.2449          